# Imports

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn. metrics import roc_auc_score, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve,roc_auc_score

In [5]:
train_1M=pd.read_csv('../data/train_1M_with_features_v1.csv')

# B0 model

In [6]:
X=train_1M.query('content_type_id==0')[['qstats_answered_correctly','user_avg_score_cum']]
y=train_1M.query('content_type_id==0')['answered_correctly']

In [10]:
log = LogisticRegression(solver='lbfgs', max_iter=1000)
y_predic=cross_val_predict(log,X,y,method='predict_proba',cv=10,verbose=1,)
y_predic_proba1=y_predic[:,1]  #on prend la 2e colonne, celle des probas de la classe 1
roc_auc_score(y,y_predic_proba1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   18.9s finished


0.7467683126290192

# B0 ++ avec qq features

In [11]:
selected_features=['qstats_answered_correctly','user_avg_score_cum','already_seen','user_personalized_qstat_knowing_had_explanation_or_not','at_least_one_lesson']
X=train_1M.query('content_type_id==0')[selected_features]
y=train_1M.query('content_type_id==0')['answered_correctly']
X.isna().sum()

qstats_answered_correctly                                 0
user_avg_score_cum                                        0
already_seen                                              0
user_personalized_qstat_knowing_had_explanation_or_not    0
at_least_one_lesson                                       0
dtype: int64

In [12]:
log = LogisticRegression(solver='lbfgs', max_iter=1000)
y_predic=cross_val_predict(log,X,y,method='predict_proba',cv=10)
y_predic_proba1=y_predic[:,1]  #on prend la 2e colonne, celle des probas de la classe 1
roc_auc_score(y,y_predic_proba1)

0.7575087237925182